# SVM for german Corpora with k-fold
Source

[1] sklearn https://scikit-learn.org/stable/ <br>
[2] re https://docs.python.org/3/library/re.html <br>
[3] pandas https://pandas.pydata.org/ <br>
[4] time https://docs.python.org/3/library/time.html <br>
[5] numpy https://numpy.org/ <br>

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html <br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html <br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold <br>

In [1]:
import sklearn
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold,train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import re
import pandas as pd
import time
import numpy as np


## Splitting Data

Splits the data in test and training set. For k=10 Video= https://www.youtube.com/watch?v=gJo0uNL-5Qw

In [2]:
def splitData():
    global train_index, test_index
    #y = data.sentiment.values 
    #x_train, x_test, y_train, y_test = train_test_split(data.preprocessedData.values, y, 
    #                                                    stratify=y, 
    #                                                    random_state=1, 
    #                                                    test_size=0.3, shuffle=True)
    #kf=KFold(n_splits=4)
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2)
    iteration=0
    #print(data.head())
    data["sentiment"].replace(to_replace="positive", value=1, regex=True, inplace=True)
    data["sentiment"].replace(to_replace="negative", value=0, regex=True, inplace=True)
    data["sentiment"].replace(to_replace="neutral", value=2, regex=True, inplace=True)
    #print(X)
    for train_index, test_index in skf.split(data["preprocessedData"],data["sentiment"]):
        iteration=iteration+1
        startTime=time.time()
        #print("TRAIN:", train_index, "TEST:", test_index)
        #x_train, x_test, y_train, y_test=data.preprocessedData.values[train_index],data.preprocessedData.values[test_index],data.sentiment.values[train_index],data.sentiment.values[test_index]
        
        x_train, x_test, y_train, y_test=data.preprocessedData.values[train_index],data.preprocessedData.values[test_index],data.sentiment.values[train_index],data.sentiment.values[test_index]
        vectorizer= CountVectorizer()
        
        train_vec=vectorizer.fit_transform(x_train)
        test_vec=vectorizer.transform(x_test)
        
        train(train_vec,test_vec, y_train,y_test,startTime,iteration)

## Train Model

In [3]:
def train(train_vec,test_vec,y_train,y_test,startTime,iteration):
    global test_set
    mnb =MultinomialNB()
    prob=mnb.fit(train_vec,y_train)
    test_set=mnb.predict(test_vec)
    
    #print(test_set)
    #clas_svm=svm.SVC(kernel="linear",probability=True, cache_size=1800)
    #prob=clas_svm.fit(train_vec,y_train).predict(test_vec)

    #test_set=clas_svm.predict(test_vec)
    getData(test_set,y_train,y_test,startTime,iteration)

## Get Data

In [4]:
def getData(test_set,y_train,y_test,startTime,iteration):
    
    #global corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary
    corporaType=""
    shortcut=""
    name=""
    totalTextUnits=0
    totalTestUnits=0
    totalTrainUnits=0
    accuracy=0
    f1_macro=0
    precision_macro=0
    recall_macro=0
    f1_micro=0
    precision_micro=0
    recall_micro=0
    matrix=[]
    f1_binary=0
    precision_binary=0
    recall_binary=0

    splitString=re.split("_|/",file)
    corporaType=splitString[4]
    shortcut=splitString[5]
    name=splitString[6]

    totalTime=time.time()-startTime

    totalTextUnits=len(data.index)
    totalTestUnits=len(test_set)
    totalTrainUnits=len(y_train)
    
    accuracy=accuracy_score(y_test, test_set)
    #print("Type", corporaType)
    if corporaType == "Ternary":
        f1_macro=f1_score(y_test, test_set, average="macro")
        precision_macro=precision_score(y_test, test_set, average="macro")
        recall_macro=recall_score(y_test, test_set, average="macro")

        f1_micro=f1_score(y_test, test_set, average="micro")
        precision_micro=precision_score(y_test, test_set, average="micro")
        recall_micro=recall_score(y_test, test_set, average="micro")

        matrix=confusion_matrix(y_test, test_set, labels=[0,1,2])
        matrixFlat=convertMatrix(matrix)
        print(y_test)
        print("-----------")
        print(test_set)
        target_names = [0,1,2]
        classificationReport=classification_report(y_test, test_set, target_names=target_names, output_dict=True)
        #ar=te.flattern()

    if corporaType == "Binary":
        #print("Inside")
        f1_binary=f1_score(y_test, test_set, average="binary",pos_label=1)
        precision_binary=precision_score(y_test, test_set, average="binary",pos_label=1)
        recall_binary=recall_score(y_test, test_set, average="binary",pos_label=1)
        
        matrix=confusion_matrix(y_test, test_set, labels=[1,0])
        
        matrixFlat=convertMatrix(matrix)
        target_names = [0,1]
        classificationReport=classification_report(y_test, test_set, target_names=target_names, output_dict=True)
        #print(matrixFlat)
    saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,y_test,test_set,startTime,iteration,matrixFlat,classificationReport)


In [5]:
def convertMatrix(matrix):
    global flatMatrix
    array=[]
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            array.append(matrix[i][j])
    flatMatrix = np.array(array)
    #print("Funktion",flatMatrix)
    return flatMatrix

In [6]:
def saveData(corporaType,shortcut,name,totalTime,totalTextUnits,totalTestUnits,totalTrainUnits,accuracy,f1_macro,precision_macro,recall_macro,f1_micro,precision_micro,recall_micro,matrix,f1_binary,precision_binary,recall_binary,y_test,test_set,startTime,iteration, matrixFlat,classificationReport):
    #print(matrixFlat)
    df_svm_data=pd.read_csv("NaiveBayesDataKFold.tsv", sep="\t")
    df_svm_data_full=pd.read_csv("NaiveBayesDataKFoldFull.tsv", sep="\t")

    #df=pd.DataFrame(columns=["Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
    #          "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
    #        "F1 Micro","F1 Binary","Matrix"])

    allData={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat}

    allDataFull={"Iteration":iteration,"Shortcut":shortcut,"Name":name,"Type":corporaType,"Time":totalTime,"Total Length":totalTextUnits,"Training Set":totalTrainUnits,"Test Set":totalTestUnits,"Accuracy":accuracy,"Precision Macro":precision_macro,
             "Precision Micro":precision_micro,"Precision Binary":precision_binary,"Recall Macro":recall_macro,"Recall Micro":recall_micro,"Recall Binary":recall_binary,"F1 Macro":f1_macro,
            "F1 Micro":f1_micro,"F1 Binary":f1_binary,"Matrix":matrixFlat,"Train Set Full":y_test,"Test Set Full":test_set}

    
    reportDict=transformReport(classificationReport)
    allData.update(reportDict)
    allDataFull.update(reportDict)
    #print(allData)
    df_new_data=pd.DataFrame([allData])
    df_new_data_full=pd.DataFrame([allDataFull])

    finalData_svm=pd.concat([df_svm_data,df_new_data])
    #print(finalData_svm)
    finalData_svm_full=pd.concat([df_svm_data_full,df_new_data_full])
    
    finalData_svm=finalData_svm[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]

    finalData_svm_full=finalData_svm_full[["Iteration","Shortcut","Name","Type","Time","Total Length","Training Set","Test Set","Accuracy","Precision Macro",
              "Precision Micro","Precision Binary","Recall Macro","Recall Micro","Recall Binary","F1 Macro",
            "F1 Micro","F1 Binary","Matrix","Train Set Full","Test Set Full","0 precision","0 recall","0 f1-score",
            "0 support","1 precision","1 recall","1 f1-score","1 support","accuracy accuracy",
            "macro avg precision","macro avg recall","macro avg f1-score","macro avg support","weighted avg precision","weighted avg recall",
            "weighted avg f1-score","weighted avg support"]]
    
    finalData_svm.to_csv("NaiveBayesDataKFold.tsv", sep="\t",index=False)
    finalData_svm_full.to_csv("NaiveBayesDataKFoldFull.tsv", sep="\t",index=False)
# ,"neutral f1-score","neutral support","neutral precision","neutral recall"
# "neutral precision","neutral recall","neutral f1-score","neutral support",
#	neutral precision	neutral recall	neutral f1-score	neutral support
#	neutral precision	neutral recall	neutral f1-score	neutral support

In [7]:
def transformReport(classificationReport):
    newDict={}
    for key in classificationReport.keys():
        mainName=str(key)
        if type(classificationReport[key]) != dict:
            name=mainName+" "+key
            newDict[name]=classificationReport[key]
            #print(key, "->",classificationReport[key],"-",)
            
        if type(classificationReport[key]) == dict:
            for k in classificationReport[key].keys():
                name=mainName+" "+k
                newDict[name]=classificationReport[key][k]
                #print(k,"->",classificationReport[key][k])
    #print("--------------")
    #print("DICT:",newDict)
    return newDict

In [8]:
#def main():
 #   global data, file
  #  print("Start")
   # file="../../preprocessed/ternary/MI01_mlsa_Preprocessed_ternary.tsv"
   # data=pd.read_csv(file, sep="\t")
    #splitData()
    #print("Finish")

#main()

In [9]:
filesBinaryTest=["../../Corpora/Preprocessed/Binary/LT01_gnd_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/LT02_speechLessing_Preprocessed_binary.tsv"]

filesBinaryAgain=["../../Corpora/Preprocessed/Binary/SM05_ironycorpus_Preprocessed_binary.tsv"]

filesBinary=["../../Corpora/Preprocessed/Binary/LT01_gnd_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/LT02_speechLessing_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/LT03_historicplays_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/MI01_mlsa_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/MI02_germeval_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/MI03_corpusRauh_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/NA01_gersen_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/NA02_gerom_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/NA03_ompc_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/RE01_usage_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/RE03_critics_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM01_sb10k_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM02_potts_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM03_multiSe_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM04_gertwittersent_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM05_ironycorpus_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/SM06_celeb_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/RE02_scare_Preprocessed_binary_balanced.tsv",
       "../../Corpora/Preprocessed/Binary/RE04_filmstarts_Preprocessed_binary.tsv",
       "../../Corpora/Preprocessed/Binary/RE05_amazonreviews_Preprocessed_binary_balanced.tsv"]

filesTernaryTest=["../../Corpora/Preprocessed/Ternary/LT01_gnd_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/LT02_speechLessing_Preprocessed_ternary.tsv",
        "../../Corpora/Preprocessed/Ternary/SM05_ironycorpus_Preprocessed_ternary.tsv"]

filesTernaryAgain=["../../Corpora/Preprocessed/Ternary/LT01_gnd_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/NA02_gerom_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/NA03_ompc_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/RE01_usage_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM05_ironycorpus_Preprocessed_ternary.tsv"]

filesTernary=["../../Corpora/Preprocessed/Ternary/LT01_gnd_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/LT02_speechLessing_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/MI01_mlsa_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/MI02_germeval_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/MI03_corpusRauh_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/NA01_gersen_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/NA02_gerom_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/NA03_ompc_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/RE01_usage_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/RE03_critics_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM01_sb10k_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM02_potts_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM03_multiSe_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM04_gertwittersent_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/SM05_ironycorpus_Preprocessed_ternary.tsv",       
       "../../Corpora/Preprocessed/Ternary/SM06_celeb_Preprocessed_ternary.tsv",
       "../../Corpora/Preprocessed/Ternary/RE02_scare_Preprocessed_ternary_balanced.tsv",
       "../../Corpora/Preprocessed/Ternary/RE04_filmstarts_Preprocessed_ternary_all.tsv"]

def main():
    global data, file
    for file in filesBinary:
        print("Start for corpora: ",file)
        #file="../../Corpora/Preprocessed/Binary/MI01_mlsa_Preprocessed_ternary.tsv"
        data=pd.read_csv(file, sep="\t")
        splitData()
        print("Finish")

main()

Start for corpora:  ../../Corpora/Preprocessed/Binary/LT01_gnd_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/LT02_speechLessing_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/LT03_historicplays_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/MI01_mlsa_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/MI02_germeval_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/MI03_corpusRauh_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/NA01_gersen_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/NA02_gerom_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/NA03_ompc_Preprocessed_binary.tsv


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: Unde

Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/RE01_usage_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/RE03_critics_Preprocessed_binary.tsv


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif

Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM01_sb10k_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM02_potts_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM03_multiSe_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM04_gertwittersent_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM05_ironycorpus_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/SM06_celeb_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/RE02_scare_Preprocessed_binary_balanced.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/RE04_filmstarts_Preprocessed_binary.tsv
Finish
Start for corpora:  ../../Corpora/Preprocessed/Binary/RE05_amazonreviews_Preprocessed_binary_balanced.tsv
Finish


Binary maybe scare?
RE03 UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
Na01 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))  
  
  
  NA02 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
  Ternary
  
  RE03 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
  
  NA01 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
  RE01 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
  SM06 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  
  NA02 Error  The number of classes has to be greater than one; got 1 class


### New Error Message at (Binary):
NA03 /usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
### Ternary
Na02
Na03
Re01